In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# NLP: Analyzing Review Text


## A note on scoring

## Download and parse the data


In [45]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review_reduced.json.gz'

The training data are a series of JSON objects, in a Gzipped file. Python supports Gzipped files natively: [`gzip.open`](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [`ujson`](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [46]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review_reduced.json.gz') as f:
    data = [json.loads(line) for line in f]

In [47]:
data[0]

{'votes': {'funny': 0, 'useful': 0, 'cool': 0},
 'user_id': 'Qrs3EICADUKNFoUq2iHStA',
 'review_id': '_ePLBPrkrf4bhyiKWEn4Qg',
 'stars': 1,
 'date': '2013-04-19',
 'text': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office ha

Scikit Learn will want the labels in a separate data structure, so let's pull those out now.

In [5]:
stars = [row['stars'] for row in data]

## Building models


# Questions


## bag_of_words_model

Build a linear model predicting the star rating based on the count of the words in each document (bag-of-words model).  Use a [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) or [`HashingVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) to produce a feature matrix giving the counts of each word in each review.  Feed this in to linear model, such as `Ridge` or `SGDRegressor`, to predict the number of stars from each review.

**Hints**:
1. Don't forget to use tokenization!  This is important for good performance but it is also the most expensive step.  Try vectorizing as a first initial step and then running grid-search and cross-validation only on of this pre-processed data.  `CountVectorizer` has to memorize the mapping between words and the index to which it is assigned.  This is linear in the size of the vocabulary.  The `HashingVectorizer` does not have to remember this mapping and will lead to much smaller models.

```python
from sklearn.feature_extraction.text import CountVectorizer

text = [row['text'] for row in data]
X = CountVectorizer().fit_transform(text)

# Now, this can be run with many different parameters
# without needing to retrain the vectorizer:
model.fit(X, stars, hyperparameter=something)
```

2. Try choosing different values for `min_df` (minimum document frequency cutoff) and `max_df` in `CountVectorizer`.  Setting `min_df` to zero admits rare words which might only appear once in the entire corpus.  This is both prone to overfitting and makes your data unmanageably large.  Don't forget to use cross-validation or to select the right value.  Notice that `HashingVectorizer` doesn't support `min_df`  and `max_df`.  However, it's not hard to roll your own transformer that solves for these.

3. Try using [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) or [`RidgeCV`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV).  If the memory footprint is too big, try switching to [Stochastic Gradient Descent](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) You might find that even ordinary linear regression fails due to the data size.  Don't forget to use [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to determine the regularization parameter!  How do the regularization parameter `alpha` and the values of `min_df` and `max_df` from `CountVectorizer` change the answer?

4. You will likely pick up several hyperparameters between the tokenization step and the regularization of the estimator.  While is is more strictly correct to do a grid search over all of them at once, this can take a long time. Quite often, doing a grid search over a single hyperparameter at a time can produce similar results.  Alternatively, the grid search may be done over a smaller subset of the data, as long as it is representative of the whole.

5. Finally, assemble a pipeline that will transform the data from records all the way to predictions.  This will allow you to submit its predict method to the grader for scoring.

In [92]:
data[0]

{'votes': {'funny': 0, 'useful': 0, 'cool': 0},
 'user_id': 'Qrs3EICADUKNFoUq2iHStA',
 'review_id': '_ePLBPrkrf4bhyiKWEn4Qg',
 'stars': 1,
 'date': '2013-04-19',
 'text': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office ha

In [6]:
from sklearn.pipeline import Pipeline
from sklearn import base
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
import time
from sklearn.model_selection import train_test_split

class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):    
    def __init__(self, col_name):
        self.col_name = col_name
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        selected = []
        for row in X:
            selected.append(row[self.col_name])
        return selected

text = ColumnSelectTransformer(col_name='text').transform(data)

In [74]:
#X = CountVectorizer(min_df=0.002, max_df=0.6, stop_words='english').fit_transform(text)
X = HashingVectorizer(stop_words='english').fit_transform(text)

In [15]:
import numpy as np
y = np.array(stars)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=9527)
start = time.time()
ridge = RidgeCV(alphas=[1,10], cv=3)
ridge.fit(X_train, y_train)
print (ridge.score(X_test,y_test))
print ('ET:', time.time()-start)

0.6064543490546254
ET: 55.463743925094604


In [82]:
ridge.alpha_

1

In [50]:
bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    ('cs', ColumnSelectTransformer(col_name='text')),('vectorizer', HashingVectorizer(stop_words='english')), 
    ('regressor', Ridge(alpha=1))])
bag_of_words_est.fit(data, stars)

Pipeline(memory=None,
     steps=[('cs', ColumnSelectTransformer(col_name='text')), ('vectorizer', HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

## normalized_model

Normalization is key for good linear regression. Previously, we used the count as the normalization scheme.  Add in a normalization transformer to your pipeline to improve the score.  Try some of these:

1. You can use the "does this word present in this document" as a normalization scheme, which means the values are always 1 or 0.  So we give no additional weight to the presence of the word multiple times.

2. Try using the log of the number of counts (or more precisely, $log(x+1)$). This is often used because we want the repeated presence of a word to count for more but not have that effect tapper off.

3. [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a common normalization scheme used in text processing.  Use the [`TfidfTransformer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). There are options for using `idf` and taking the logarithm of `tf`.  Do these significantly affect the result?

Finally, if you can't decide which one is better, don't forget that you can combine models with a linear regression.

In [87]:
X_idf = TfidfTransformer().fit_transform(X)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X_idf,y,test_size=0.3,random_state=9527)
start = time.time()
ridge = RidgeCV(alphas=[1,10], cv=3)
ridge.fit(X_train, y_train)
print (ridge.score(X_test,y_test))
print ('ET:', time.time()-start)

0.5913948215389855
ET: 56.083779096603394


In [90]:
from sklearn.feature_extraction.text import TfidfTransformer
normalized_est = Pipeline([
    ('cs', ColumnSelectTransformer(col_name='text')),('vectorizer', HashingVectorizer(stop_words='english')),('tfidf', TfidfTransformer(norm='l2')),
    ('regressor', Ridge(alpha=1))])
normalized_est.fit(data, stars)

Pipeline(memory=None,
     steps=[('cs', ColumnSelectTransformer(col_name='text')), ('vectorizer', HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

## bigram_model

In a bigram model, we'll consider both single words and pairs of consecutive words that appear.  This is going to be a much higher dimensional problem (large $p$) so you should be careful about overfitting.

Sometimes, reducing the dimension can be useful.  Because we are dealing with a sparse matrix, we have to use [`TruncatedSVD`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD).  If we reduce the dimensions, we can use a more sophisticated models than linear ones.

As before, memory problems can crop up due to the engineering constraints. Playing with the number of features, using the `HashingVectorizer`, incorporating `min_df` and `max_df` limits, and handling stop-words in some way are all methods of addressing this issue. If you are using `CountVectorizer`, it is possible to run it with a fixed vocabulary (based on a training run, for instance). Check the documentation.

**A side note on multi-stage model evaluation:** When your model consists of a pipeline with several stages, it can be worthwhile to evaluate which parts of the pipeline have the greatest impact on the overall accuracy (or other metric) of the model. This allows you to focus your efforts on improving the important algorithms, and leaving the rest "good enough".

One way to accomplish this is through ceiling analysis, which can be useful when you have a training set with ground truth values at each stage. Let's say you're training a model to extract image captions from websites and return a list of names that were in the caption. Your overall accuracy at some point reaches 70%. You can try manually giving the model what you know are the correct image captions from the training set, and see how the accuracy improves (maybe up to 75%). Alternatively, giving the model the perfect name parsing for each caption increases accuracy to 90%. This indicates that the name parsing is a much more promising target for further work, and the caption extraction is a relatively smaller factor in the overall performance.

If you don't know the right answers at different stages of the pipeline, you can still evaluate how important different parts of the model are to its performance by changing or removing certain steps while keeping everything else constant. You might try this kind of analysis to determine how important adding stopwords and stemming to your NLP model actually is, and how that importance changes with parameters like the number of features.

In [20]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
import time
start = time.time()
bigram_est = Pipeline([
    ('cs', ColumnSelectTransformer(col_name='text')),('vectorizer', HashingVectorizer(stop_words='english', ngram_range=(1,2))),
    ('trunc', TruncatedSVD(n_components=300, algorithm='randomized', n_iter=5, random_state=9527)),('regressor', Ridge(alpha=1))])
bigram_est.fit(data, stars)
print ('ET:', time.time()-start)

ET: 596.366409778595


## food_bigrams

Look over all reviews of restaurants.  You can determine which businesses are restaurants by looking in the `yelp_train_academic_dataset_business.json.gz` file from the ml project or downloaded below.

In [24]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review.json.gz'

In [49]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f]

In [50]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]

Each row of this file corresponds to a single business.  The category key gives a list of categories for each; take all where "Restaurants" appears.

In [52]:
restaurant_ids = [x['business_id'] for x in business_data if 'Restaurants' in x['categories']]

In [53]:
assert len(restaurant_ids) == 12876

The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [29]:
restaurant_reviews = [x['text'] for x in data if x['business_id'] in restaurant_ids]

In [55]:
len(restaurant_reviews)

574278

In [56]:
#ignore this if using the updated data file
#assert len(restaurant_reviews) == 143361

In [54]:
import pickle
pickle.dump(restaurant_reviews, open('restaurant_reviews.p', 'wb'))

We want to find collocations --- that is, bigrams that are "special" and appear more often than you'd expect from chance. We can think of the corpus as defining an empirical distribution over all *n*-grams.  We can find word pairs that are unlikely to occur consecutively based on the underlying probability of their words. Mathematically, if $p(w)$ be the probability of a word $w$ and $p(w_1 w_2)$ is the probability of the bigram $w_1 w_2$, then we want to look at word pairs $w_1 w_2$ where the statistic

  $$ \frac{p(w_1 w_2)}{p(w_1) p(w_2)} $$

is high.  Return the top 100 (mostly food) bigrams with this statistic with the 'right' prior factor (see below).

Estimating the probabilities is simply a matter of counting, and there are number of approaches that will work.  One is to use one of the tokenizers to count up how many times each word and each bigram appears in each review, and then sum those up over all reviews.  You might want to know that the `CountVectorizer` has a `.get_feature_names()` method which gives the string associated with each column.  (Question for thought: Why doesn't the `HashingVectorizer` have a similar method?)

*Questions:* This statistic is a ratio and problematic when the denominator is small.  We can fix this by applying Bayesian smoothing to $p(w)$ (i.e. mixing the empirical distribution with the uniform distribution over the vocabulary).

1. How does changing this smoothing parameter affect the word pairs you get qualitatively?

2. We can interpret the smoothing parameter as adding a constant number of occurrences of each word to our distribution.  Does this help you determine set a reasonable value for this 'prior factor'?

3. For fun: also check out [Amazon's Statistically Improbable Phrases](http://en.wikipedia.org/wiki/Statistically_Improbable_Phrases).

*Implementation note:*
As you adjust the size of the Bayesian smoothing parameter, you will notice first nonsense phrases being removed and then legitimate bigrams being removed, leaving you with only generic bigrams.  The goal is to find a value of the smoothing parameter between these two transitions.

The reference solution is not an aggressive filterer: it errors in favor of leaving apparently nonsensical words. On further consideration, many of these are actually somewhat meaningful. The smoothing parameter chosen in the reference solution is equivalent to giving each word 30 previous appearances prior to considering this data.  This was chosen by generating a list of bigrams for a range of smoothing parameters and seeing how many of the bigrams were shared between neighboring values.  When the shared fraction reached 95%, we judged the solution to have converged.  Note that `min_df` should not be set too high, where it could exclude these borderline words.

In [67]:
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), stop_words='english',min_df=7)
words_vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english',min_df=5)

bigrams = bigram_vectorizer.fit_transform(restaurant_reviews)
words = words_vectorizer.fit_transform(restaurant_reviews)

In [83]:
bi_cnt = bigrams.sum(axis=0)
w_cnt = words.sum(axis=0)

In [137]:
bigram_dict = dict(zip(bigram_vectorizer.get_feature_names(), np.array(bi_cnt)[0]))
word_dict = dict(zip(words_vectorizer.get_feature_names(), np.array(w_cnt)[0]))

In [121]:
import pandas as pd
import numpy as np

In [91]:
calc_df = pd.DataFrame()
calc_df['bigram'] = bigram_vectorizer.get_feature_names()
calc_df['word1'] = [x.split()[0] for x in bigram_vectorizer.get_feature_names()]
calc_df['word2'] = [x.split()[1] for x in bigram_vectorizer.get_feature_names()]

In [147]:
word1_cnt = [word_dict.get(calc_df.loc[i]['word1'],0) for i in range(calc_df.shape[0])]
word2_cnt = [word_dict.get(calc_df.loc[i]['word2'],0) for i in range(calc_df.shape[0])]

In [151]:
#calc_df['word1_cnt'] = word1_cnt
#calc_df['word2_cnt'] = word2_cnt
calc_df['bi_cnt'] = np.array(bi_cnt)[0]

In [153]:
df = calc_df[calc_df['bi_cnt'] >= 30]

In [176]:
df['prob'] = df['bi_cnt']/((df['word1_cnt']+90)*(df['word2_cnt']+90))

/opt/conda/envs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [177]:
df.sort_values(by='prob',ascending=False,inplace=True)
df.head(10)

/opt/conda/envs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,bigram,word1,word2,word1_cnt,word2_cnt,bi_cnt,prob
177101,ezzyujdouig4p gyb3pv_a,ezzyujdouig4p,gyb3pv_a,92,92,92,0.002777
243991,hodge podge,hodge,podge,55,54,54,0.002586
243324,himal chuli,himal,chuli,57,56,54,0.002516
244013,hoity toity,hoity,toity,61,55,55,0.002512
432743,roka akor,roka,akor,48,46,46,0.002451
177102,f_5_unx wrafcxuakbzrdw,f_5_unx,wrafcxuakbzrdw,44,44,44,0.002450
273704,knick knacks,knick,knacks,111,98,92,0.002435
419768,reina pepiada,reina,pepiada,83,70,67,0.002421
94301,cien agaves,cien,agaves,88,76,71,0.002403
38898,baskin robbins,baskin,robbins,74,73,64,0.002394


In [178]:
top100 = list(df['bigram'][:100])